In [1]:
import os
import pyxdf # xdf file importation
import numpy as np
from my_functions import *

### Ensure that the root directory is the project directory
In some cases, the root directory may not be the project directory. This is usually the case when the notebook is run from a subdirectory of the project directory.

In [2]:
# if pwd is SRC  change to root
if os.getcwd().split("/")[-1] == "SRC":
    os.chdir("..")
print("Current working directory: ", os.getcwd())

Current working directory:  c:\Users\iWiss\Documents\GitHub\Stage_Signal_Physio



## Initialization

### File selection and loading

In [3]:
# Define the xdf file path
FILENAME = "001_MolLud_20201112_1_c.xdf"
# FILENAME="020_DesMar_20211129_1_c.xdf"
# path=os.path.normpath("../DAT/Input/001_MolLud_20201112_1_c.xdf")
path = os.path.normpath("DAT/INPUT/"+FILENAME)

### Loading streams of interest

In [4]:
data, header = pyxdf.load_xdf(path, select_streams=[{'type': 'EEG'}, {
                              'type': 'Markers', 'name': 'MouseToNIC'}])

#### Storing the data of interest in two lists (markers and signal) 

In [5]:
# Selection of the streams of interest in the xdf file (data is a list of dictionaries)
for i in range(len(data)):
    #print(i)
    if data[i]["info"]["type"] == ['EEG']:
        #print(i, "est EEG")
        EEG_Stream = data[i]  # selecting EEG stream
    elif data[i]["info"]["type"] == ['Markers']:
        #print(i, "est Marker")
        Mouse_markers_Stream = data[i]  # selecting markers stream

#### Defining the channels names

In [6]:
# definition of the EEG channels' names
channels_dic = {"Channel_1": "C4",
                "Channel_2": "FC2",
                "Channel_3": "FC6",
                "Channel_4": "CP2",
                "Channel_5": "C3",
                "Channel_6": "FC1",
                "Channel_7": "FC5",
                "Channel_8": "CP1"}

### Shaping recoding parameters' data

In [7]:
Srate = EEG_Stream["info"]["effective_srate"]

# Markers start with a time relative to the execution time of the recording
Marker_times = (
    Mouse_markers_Stream["time_stamps"]-EEG_Stream["time_stamps"][0])

# selecting the marker labels
Markers_labels = Mouse_markers_Stream["time_series"]

# Creation of a 2D array Markers_times_labels: [[markers_timesstamps],[markers_labels]]
Markers_times_labels = np.column_stack((Marker_times, Markers_labels))

# format times_stamps in seconds relative to the execution time of the recording
EEG_times = EEG_Stream["time_stamps"]-EEG_Stream["time_stamps"][0]
# Amplitude of voltage recorded by each electrode of the recording set-up
EEG_raw_amplitudes = EEG_Stream["time_series"]

### Export

#### Data to be exported

In [8]:
Srate,EEG_times,EEG_raw_amplitudes

print(Srate.shape)
print(EEG_times.shape)
print(EEG_raw_amplitudes.shape)

#t=np.stack((EEG_times,EEG_raw_amplitudes),axis=1)

times_amplitudes=np.column_stack((EEG_raw_amplitudes,EEG_times))

print(times_amplitudes[:,-1]==EEG_times)


()
(221938,)
(221938, 8)
[ True  True  True ...  True  True  True]


#### Export to CSV file

In [9]:
#creating filename
originalfilename=FILENAME.replace(".xdf","_")
print(originalfilename)
exportfilename=originalfilename+str(int(Srate))+".txt"
exportfilepath=os.path.normpath("DAT/OUTPUT/"+exportfilename)
print(exportfilename)
print(exportfilepath)

001_MolLud_20201112_1_c_
001_MolLud_20201112_1_c_499.txt
DAT\OUTPUT\001_MolLud_20201112_1_c_499.txt


In [10]:
np.savetxt(exportfilepath,times_amplitudes,delimiter=",")